# Applied Data Science Capstone
This is the notebook that will contain the IBM Datascience - Applied Data Science Capstone Project

In [24]:
#! pip install pandas
#! pip install numpy
#! pip install requests
#! pip install beautifulsoup4
#! pip install geocoder
#!pip install folium

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import geocoder
import folium

In [25]:
print('Hellow Capstone Project Course!')

Hellow Capstone Project Course!


## Scrape Wikipedia for Toronto Postcodes

Extract the rows from the main table in the `List of postal codes of Canada: M Wikipedia` page.

In [3]:
wikipedia_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
postcodes = requests.get(wikipedia_url)
postcode_soup = BeautifulSoup(postcodes.content, 'html.parser')
tables = postcode_soup.find_all('table')
postcode_table = tables[0]
table_body = postcode_table.find('tbody')
rows = table_body.find_all('tr')

Iterate through the table rows and extract the test corresponding to the District + Neighbourhoods

In [4]:
district_neighbourhoods = []
for row in rows:
    cols = row.find_all('td')
    for cell in cols:
        _text = cell.text.strip()[3:] #discard first 3 chars as they are the Postcode
        if _text == 'Not assigned': #Ignore unassigned postcodes
            continue
        
        #Several entries don't conform to the majority, just hardcoding a solution.
        if _text == 'North York(Don Mills)North':
            district_neighbourhoods.append('North York(Don Mills)')
        elif _text == 'North York(Don Mills)South(Flemingdon Park)':
            district_neighbourhoods.append('North York(Don Mills)')
            district_neighbourhoods.append('South(Flemingdon Park)') 
        elif _text == 'North York(Downsview)East  (CFB Toronto)':
            district_neighbourhoods.append('North York(Downsview)')
            district_neighbourhoods.append('East(CFB Toronto)')
        else:
            district_neighbourhoods.append(_text) 

Create a list of `dicts` by spliting the district_neighbourhoods strings and create the DataFrame

In [5]:
districts = []
for i in district_neighbourhoods:
    district, neighbourhoods = i.split('(')
    neighbourhoods = neighbourhoods.replace(')', '')
    neighbourhoods = neighbourhoods.split(' / ')
    row_data = [{'district': district, 'neighbourhood': j} for j in neighbourhoods]
    districts = [*districts, *row_data]
district_df = pd.DataFrame(districts)
district_df.drop_duplicates(inplace=True)
print(district_df.shape)
district_df.head()

(215, 2)


,district,neighbourhood
0,North York,Parkwoods
1,North York,Victoria Village
2,Downtown Toronto,Regent Park
3,Downtown Toronto,Harbourfront
4,North York,Lawrence Manor


Retrieve the latitude and longitude for each neighbourhood

In [10]:
latlng = [(geocoder.arcgis(f'{i}, Toronto, Ontario')).latlng for i in district_df['neighbourhood']]
district_df['latitude'] = [i[0] for i in latlng]
district_df['longitude'] = [i[1] for i in latlng]
district_df.head()

,district,neighbourhood,latitude,longitude
0,North York,Parkwoods,43.686588,-79.409996
1,North York,Victoria Village,43.731540,-79.314280
2,Downtown Toronto,Regent Park,43.660690,-79.360310
3,Downtown Toronto,Harbourfront,43.639230,-79.383070
4,North York,Lawrence Manor,43.722940,-79.431160


Visualise the Neighbourhood's on a map of Toronto

In [22]:
toronto = geocoder.arcgis("Toronto, Ontario")
print(f"Toronto -Latitude: {toronto.latlng[0]}, Longitude: {toronto.latlng[1]}")
map_toronto = folium.Map(location=[toronto.latlng[0], toronto.latlng[1]], zoom_start=11)
for lat, lng, district, neighbourhood in zip(district_df['latitude'], district_df['longitude'], district_df['district'], district_df['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

Toronto -Latitude: 43.648690000000045, Longitude: -79.38543999999996


In [23]:
# Foursqure details
CLIENT_ID = 'OKC2TIOLVKUWVILMC5II1EYJRQT0XB31VQ53KTD1G04HAIUQ' 
CLIENT_SECRET = 'ZVVEAZBX2OEE22F3FKOLLJ1VOADX2TA4KGOCITRVR2CNTGFQ' 
VERSION = '20180605'
LIMIT = 100

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']